In [ ]:
import chess
import numpy as np
import pandas as pd
import numexpr as ne
import swifter
import statsmodels.api as sm
import pylab
import sys

In [ ]:
df = pd.read_parquet("./random_evals_processed.parquet")
df.head(1)

In [ ]:
df_filt = df[~((df['Evaluation'] == -318) | (df['Evaluation'] == 318))]
mask = np.random.rand(len(df_filt)) < 0.8
train = df_filt[mask]
test = df_filt[~mask]

In [ ]:
from sklearn.linear_model import LinearRegression
X = list(train['Bits'])
y = list(train['Evaluation'])
reg = LinearRegression(fit_intercept = False).fit(X, y)

In [ ]:
X_test = list(test['Bits'])
y_test = list(test['Evaluation'])
reg.score(X_test, y_test)

In [ ]:
regme = LinearRegression(fit_intercept = False).fit(
    [
        [1,0,0,0,0],
        [0,1,0,0,0],
        [0,0,1,0,0],
        [0,0,0,1,0],
        [0,0,0,0,1],
    ],
    [ 1, 3, 3.5, 5, 10 ]
)
regme.score(X_test, y_test)

In [ ]:
def get_val(pc):
    os = 64 * (pc - 1)
    idx_range = range(os + 8, os + 56) if pc == chess.PAWN else range(os, os + 64)
    n = 48 if pc == chess.PAWN else 64
    v = sum(reg.coef_[idx_range]) / n
    return v

def norm_val(v):
    pv = get_val(chess.PAWN)
    return v / pv # 8 * (v - pv) / (qv - pv) + 1

def get_norm_val(pc):
    return norm_val(get_val(pc))

def print_sqs(pc):
    avg_val = get_norm_val(pc)
    print("AVG VAL: ", avg_val)

    os = 64 * (pc - 1)
    arr = reg.coef_[os : os + 64]
#     arr = [ norm_val(x) - avg_val for x in arr ]
    for i in range(0, 64):
        print(int(arr[i] * 10) / 10, end="\t")
        if i % 8 == 7:
            print()

In [ ]:
get_norm_val(chess.KING)

In [ ]:
# print sq loss
y_pred = reg.predict(X_test)
y_delta = np.absolute(y_pred - y_test)
y_dsq = y_delta * y_delta
sum(y_delta) / len(y_delta)